# CAPÍTULO 5: Redes Neuronales Artificiales y Máquinas de Vectores Soporte  


*Noemi González Lois*

**MÓDULOS**

In [1]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
import sklearn

# Ignorar advertencias de futuras versiones
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

CARGAMOS Y VISUALIZAMOS LOS DATOS DE LA BASE

Cargamos el **subconjunto de train y test** de la tarea 1:

In [2]:
atributos=['BI-RADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity']
datos_train=pd.read_csv('SubconjuntoTraining.csv', delimiter=',')
datos_test=pd.read_csv('SubconjuntoTest.csv', delimiter=',')

Establecemos como target de nuestro **clasificador** la variable 'Severity' y como datos el resto de las variables.

Creamos y normalizamos todos los subconjuntos de datos:

In [3]:
norm=MinMaxScaler()

# VALIDATION
target=datos_train.iloc[:,5:6]
_, datos_val, _, _ = train_test_split(datos_train, target, test_size=0.2)

datos_val=pd.DataFrame(norm.fit_transform(datos_val), columns=atributos)
X_val=datos_val.drop(['Severity'],axis='columns')
y_val=np.ravel(datos_val.iloc[:,5:6])

# TRAIN
datos_train=pd.DataFrame(norm.fit_transform(datos_train),columns=atributos)
X_train=datos_train.drop(['Severity'],axis='columns')
y_train=np.ravel(datos_train.iloc[:,5:6])

# TEST
datos_test=pd.DataFrame(norm.fit_transform(datos_test),columns=atributos)
X_test=datos_test.drop(['Severity'],axis='columns')
y_test=np.ravel(datos_test.iloc[:,5:6])

### PERCEPTRÓN MULTICAPA (MLP)

Diseño del MLP utilizando las características del **espacio de observaciones original**.

Los valores numéricos que representan el número de neuronas consideradas en la capa oculta son los siguientes:

In [4]:
n_hidden_1=5
n_hidden_2=50
n_hidden_3=500
n_hidden_4=5000

n_hidden=[n_hidden_1, n_hidden_2, n_hidden_3, n_hidden_4]

In [5]:
# Tabla para recoger las prestaciones en validacion
columns = ['Loss', 'Score']
comp_val = pd.DataFrame(columns=columns)

for i in n_hidden:
    
    print(f'\033[4;32mMLP clasiffier with {i} neurons in the hidden layer\033[0m')

    model = MLPClassifier(
        activation='relu', 
        batch_size=50, 
        hidden_layer_sizes=(i,), 
        learning_rate='constant',
        learning_rate_init=0.1, 
        max_iter=30, 
        shuffle=True, 
        solver='sgd', 
        tol=0.01, 
        early_stopping=True,
        validation_fraction=0.2, 
        verbose=True, 
        n_iter_no_change=5
    )
    model.fit(X_train, y_train)
    
    loss = model.loss_  
    comp_val.loc[i, ('Loss', 'Score')] = (loss, 'Verbose')


MLP clasiffier with 5 neurons in the hidden layer
Iteration 1, loss = 0.72450327
Validation score: 0.538961
Iteration 2, loss = 0.67377648
Validation score: 0.785714
Iteration 3, loss = 0.56923558
Validation score: 0.785714
Iteration 4, loss = 0.47111296
Validation score: 0.772727
Iteration 5, loss = 0.46256825
Validation score: 0.785714
Iteration 6, loss = 0.43869293
Validation score: 0.805195
Iteration 7, loss = 0.44299545
Validation score: 0.805195
Iteration 8, loss = 0.42428826
Validation score: 0.805195
Iteration 9, loss = 0.41782547
Validation score: 0.831169
Iteration 10, loss = 0.41703601
Validation score: 0.811688
Iteration 11, loss = 0.40756814
Validation score: 0.785714
Iteration 12, loss = 0.40920176
Validation score: 0.831169
Iteration 13, loss = 0.40827556
Validation score: 0.831169
Iteration 14, loss = 0.40473647
Validation score: 0.811688
Iteration 15, loss = 0.41252349
Validation score: 0.837662
Validation score did not improve more than tol=0.010000 for 5 consecutive 

La pérdida ha sido añadida a la siguiente tabla pero la accuracy de cada epoch para el conjunto de validación hay que verla en la celda anterior.

In [6]:
comp_val

,Loss,Score
5,0.412523,Verbose
50,0.411348,Verbose
500,0.426449,Verbose
5000,0.409796,Verbose


In [7]:
columns = ['Accuracy', 'Recall', 'F1-score']
comp_test = pd.DataFrame(columns=columns)

print('\033[4;32mMLP classifier with', n_hidden_2, 'neurons in the hidden layer\033[0m')

model = MLPClassifier(
    activation='relu', 
    batch_size=50, 
    hidden_layer_sizes=(n_hidden_2,), 
    learning_rate='constant',
    learning_rate_init=0.1, 
    max_iter=30, 
    shuffle=True, 
    solver='sgd', 
    tol=0.01, 
    early_stopping=True,
    validation_fraction=0.2, 
    verbose=True, 
    n_iter_no_change=5
)

model.fit(X_train, y_train)

predictions = model.predict(X_test)
cr = classification_report(y_test, predictions, output_dict=True)
acc, rec, f1 = cr['weighted avg']['precision'], cr['weighted avg']['recall'], cr['weighted avg']['f1-score']

comp_test.loc[n_hidden_2, ('Accuracy', 'Recall', 'F1-score')] = (acc, rec, f1)

MLP classifier with 50 neurons in the hidden layer
Iteration 1, loss = 0.68532790
Validation score: 0.811688
Iteration 2, loss = 0.51653609
Validation score: 0.837662
Iteration 3, loss = 0.48984952
Validation score: 0.850649
Iteration 4, loss = 0.46840009
Validation score: 0.857143
Iteration 5, loss = 0.46185325
Validation score: 0.863636
Iteration 6, loss = 0.44184976
Validation score: 0.870130
Iteration 7, loss = 0.44305721
Validation score: 0.889610
Iteration 8, loss = 0.43689345
Validation score: 0.857143
Iteration 9, loss = 0.43652965
Validation score: 0.883117
Iteration 10, loss = 0.43107620
Validation score: 0.863636
Iteration 11, loss = 0.42829368
Validation score: 0.870130
Iteration 12, loss = 0.42901004
Validation score: 0.850649
Iteration 13, loss = 0.42765128
Validation score: 0.870130
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.


El modelo generaliza bastate bien ya que tiene unas prestaciones bastante buenas, siendo las siguientes:

In [8]:
comp_test

,Accuracy,Recall,F1-score
50,0.838601,0.834197,0.834152


### MÁQUINAS DE VECTORES SOPORTE


Vamos a utilizar **SVC** (Support Vector Classification). La implementación de este clasificador se basa en libsvm (librería de vectores soporte para aprendizaje automático). Si quisierámos hacer casificación con bases de datos muy grandes y con kernel lineal, podríamos utilizar LinearSVC.

Como vamos a utilizar un **kernel gaussiano**, utilizaremos  un kernel de radial basis function (rbf)

Para la realización de este apartado vamos a utilizar la partición de **validación** para realizar validación cruzada. De esta forma, podremos ver en este subconjunto qué parámetros nos dan las mejores prestaciones para utilizarlos sobre el conjunto de test y evitar que el modelo sobreajuste.

In [9]:
columns  = ['Precision','Recall','F1-score']
comp_val = pd.DataFrame(columns=columns)

In [10]:
sigma_kernel_1='auto'
sigma_kernel_2='scale'
sigma_kernel_3=10
sigma_kernel=[sigma_kernel_1,sigma_kernel_2,sigma_kernel_3]
C1=10**(-6)
C2=1.0
C3=10**6
C=[C1,C2,C3]

In [11]:
for i in sigma_kernel:  
    for j in C:
        print(f'\033[4;35mSVM classifier with regularization parameter C={j} and sigma={i}\033[0m')

        model = svm.SVC(C=j, kernel='rbf', gamma=i, max_iter=30)
        model.fit(X_train, y_train)
        predictions_val = model.predict(X_val)

        print('Matriz de confusión validación:\n', confusion_matrix(y_val, predictions_val))
        print('\nClassification report validación:\n', classification_report(y_val, predictions_val))
        
        cr = classification_report(y_val, predictions_val, output_dict=True)
        prec, rec, f1 = cr['weighted avg']['precision'], cr['weighted avg']['recall'], cr['weighted avg']['f1-score']
        comp_val.loc['C='+str(j)+' '+'Sigma='+str(i), ('Precision', 'Recall', 'F1-score')] = (prec, rec, f1)

SVM classifier with regularization parameter C=1e-06 and sigma=auto
Matriz de confusión validación:
 [[65 17]
 [12 60]]

Classification report validación:
               precision    recall  f1-score   support

         0.0       0.84      0.79      0.82        82
         1.0       0.78      0.83      0.81        72

    accuracy                           0.81       154
   macro avg       0.81      0.81      0.81       154
weighted avg       0.81      0.81      0.81       154

SVM classifier with regularization parameter C=1.0 and sigma=auto
Matriz de confusión validación:
 [[65 17]
 [12 60]]

Classification report validación:
               precision    recall  f1-score   support

         0.0       0.84      0.79      0.82        82
         1.0       0.78      0.83      0.81        72

    accuracy                           0.81       154
   macro avg       0.81      0.81      0.81       154
weighted avg       0.81      0.81      0.81       154

SVM classifier with regularization p

c:\Users\Noemi\anaconda3\envs\uni\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Noemi\anaconda3\envs\uni\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Noemi\anaconda3\envs\uni\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Noemi\anaconda3\envs\uni\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\Noemi\anaconda3\envs\uni\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning

In [12]:
comp_val

,Precision,Recall,F1-score
C=1e-06 Sigma=auto,0.813797,0.811688,0.811887
C=1.0 Sigma=auto,0.813797,0.811688,0.811887
C=1000000 Sigma=auto,0.798135,0.772727,0.770502
C=1e-06 Sigma=scale,0.791259,0.785714,0.785796
C=1.0 Sigma=scale,0.821034,0.818182,0.818366
C=1000000 Sigma=scale,0.726353,0.707792,0.696424
C=1e-06 Sigma=10,0.633117,0.532468,0.460731
C=1.0 Sigma=10,0.772667,0.766234,0.762839
C=1000000 Sigma=10,0.751851,0.733766,0.724836


Observamos que con valores más altos de C, las prestaciones son más bajas. Esto nos indica que nuestros datos no son linealmente separables.

El valor ‘scale’ para el parámetro sigma, junto con un C muy pequeño es lo que nos da mejores prestaciones para nuestro conjunto de datos. 

Una vez elegidos los **mejores parámetros** de C y de sigma, los utilizamos para el **subconjunto test**:

In [13]:
model = svm.SVC(C=10**(-6), kernel='rbf', gamma='scale', max_iter=30, verbose=True)
model.fit(X_train, y_train)
predictions_test = model.predict(X_test)

print('Matriz de confusión test:\n', confusion_matrix(y_test,predictions_test))
print('\nClasification report test:\n', classification_report(y_test,predictions_test))

[LibSVM]Matriz de confusión test:
 [[79 22]
 [ 9 83]]

Clasification report test:
               precision    recall  f1-score   support

         0.0       0.90      0.78      0.84       101
         1.0       0.79      0.90      0.84        92

    accuracy                           0.84       193
   macro avg       0.84      0.84      0.84       193
weighted avg       0.85      0.84      0.84       193



c:\Users\Noemi\anaconda3\envs\uni\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


### COMPARACIÓN DE RESULTADOS

Considero más adecuado el modelo SVM ya que sirve para separar de manera linela datos no linealmete separables inicialmente y porque además generaliza mejor.

Para nuestra base de datos resulta más adecuado trabajar con los datos originales en lugar de los transformados al espacio de PCA. Esto se debe a que tenemos pocos atributos y son todos relevantes y no redundantes.


MEJORAS

Para mejorar la arquitectura MLP podríamos probar a implementar dropout para inactivar algunas neuronas y reducir así el sobreajuste, utilizar como función de coste la entropía cruzada o utilizar como función de activación la leaky ReLU.

Para mejorar la arquitectura SVM podríamos probar con otras funciones kernel.